In [1]:
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Input, Add, Reshape,Conv2D,MaxPooling2D,ZeroPadding2D,experimental,AveragePooling2D,Concatenate,Dropout,Activation
from matplotlib import pyplot as plt
from tensorflow.keras.models import Model
import numpy as np
import keras
from keras import layers
from PIL import Image
import cv2
from skimage.feature import hog
from sklearn import svm
from sklearn.metrics import accuracy_score
from skimage.io import imread, imshow
from skimage.transform import resize
from skimage.feature import hog
from skimage import exposure
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [2]:
import os
import numpy as np
#import cPickle
import _pickle as cPickle
import pandas as pd 
from numpy import argmax
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.metrics import confusion_matrix

import tensorflow.python.keras
from tensorflow.python.keras.models import Sequential 
from tensorflow.python.keras.models import model_from_json
from tensorflow.python.keras.layers import Dense, Activation, Dropout, Flatten, BatchNormalization
from tensorflow.python.keras.layers import Conv2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras import backend as K
from keras import regularizers
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.callbacks import ModelCheckpoint

In [23]:
a = plt.imread("DermNet/archive/test/Acne and Rosacea Photos/07PerioralDermEye.jpg")

In [24]:
a.shape

(472, 720, 3)

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    'DermNet/archive/train',
    target_size=(470, 720),
    batch_size=8,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    'DermNet/archive/test/',
    target_size=(470, 720),
    batch_size=8,
    class_mode='categorical')

Found 15557 images belonging to 23 classes.
Found 4002 images belonging to 23 classes.


In [4]:
weight_decay = 1e-4
model = Sequential()
model.add(Conv2D(32, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay), input_shape=(470,720,3)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(32, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(64, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(64, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))

model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())
model.add(Dense(23, activation='softmax'))

In [ ]:
batch_size = 16
checkpoint_filepath = 'DERM_DNN'
model_checkpoint_callback = ModelCheckpoint(filepath=checkpoint_filepath,
                                            monitor='val_accuracy',mode='max',
                                            save_best_only=True)
opt_rms = tensorflow.keras.optimizers.RMSprop(lr=0.001,decay=1e-6)
model.compile(loss='categorical_crossentropy', optimizer=opt_rms, metrics=['accuracy'])
history = model.fit_generator(train_generator,epochs=100,verbose=2, callbacks=[model_checkpoint_callback])

Epoch 1/100


In [ ]:
def pre_trained_model():
    input = Input((470,720,3))
    c1l1  = Conv2D(32,3,(1,1),padding = 'same',activation='relu')(input)
    c1l2  = Conv2D(32,3,(1,1),padding = 'same',activation='relu')(c1l1)
    ces2  = Add()([c1l2, c1l1])
    c1l3  = Conv2D(32,3,(1,1),padding = 'same',activation='relu')(ces2)
    ces3  = Add()([c1l3, ces2])
    ces3  = Flatten()(ces3)
    ces3  = Dense(23)(ces3)
    ces3  = Activation('softmax')(ces3)
    return Model(inputs=input, outputs=ces3)

In [ ]:
pretrained_model = pre_trained_model()
pretrained_model.compile(loss='categorical_crossentropy', optimizer=opt_rms, metrics=['accuracy'])
checkpoint_filepath = 'DERM_shallowNN'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath,
                                                               monitor='val_accuracy',mode='max',
                                                               save_best_only=True)
history = pretrained_model.fit_generator(train_generator,epochs = 100,callbacks = [model_checkpoint_callback])